# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    total = 0
    for index, row in data.iterrows():
        total += (row["Price in 1 Year"] - P) / P * row["Probability"]
    
    return total

Write the function ```volatility```, which calculates the volatility of the stock, given the current stock price ```P```.

In [4]:
def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    total = 0

    for index, row in data.iterrows():
        total += (((row["Price in 1 Year"] - P) / P - exp_return(data, P))** 2) * row["Probability"]
    arr = np.array(total)
    arr2 = np.sqrt(arr)
    return arr2

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [5]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    arr = np.array(len(data))
    arr2 = np.sqrt(arr)
    std_err = volatility(data,P) / arr2
    lower = exp_return(data, P) - 1.96 * std_err
    upper = exp_return(data, P) + 1.96 * std_err
    return [lower,upper]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [6]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    return (rf + beta * (market_return - rf))

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [7]:
def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    df = pd.read_csv(path)
    df = df[pd.to_numeric(df['RET'], errors='coerce').notnull()] # removes rows with non number RET

    # filter singletons
    groups = df.groupby(["PERMNO"], group_keys = False)
    filtered = groups.filter(lambda x: len(x) >= 500).reset_index().drop(['index'], axis = 1)

    return filtered

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [8]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    output = pd.DataFrame(data = {
        "CUSIP" : [],
        "Average Return" : []
    })

    counter = 0
    total = 0
    CUSIP = data["CUSIP"][0]
    for index, row in data.iterrows():
        if (row["CUSIP"] != CUSIP):
            output.loc[len(output.index)] = [CUSIP, total / counter]
            counter = 0
            total = 0
            CUSIP = row["CUSIP"]
        total += float(row["RET"])
        counter += 1
    output.loc[len(output.index)] = [CUSIP, total / counter]
    return output


In [9]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    
    output = pd.DataFrame(data = {
        "CUSIP" : [],
        "Volatility" : []
    })
    
    avg_returns = avg_return(data)
    counter = 0
    total = 0
    CUSIP = data["CUSIP"][0]
    securities = 0
    for index, row in data.iterrows():
        if (row["CUSIP"] != CUSIP):
            var = np.array(total / (counter - 1))
            vol = np.sqrt(var)
            output.loc[len(output.index)] = [CUSIP, vol]
            counter = 0
            total = 0
            CUSIP = row["CUSIP"]
            securities += 1
        total += (float(row["RET"]) - avg_returns.loc[securities, "Average Return"])**2
        counter += 1
    # securities += 1
    var = np.array(total / (counter - 1))
    vol = np.sqrt(var)
    output.loc[len(output.index)] = [CUSIP, vol]
    return output

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [10]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    df = pd.merge(ret_data, vol_data, how = 'inner', on = ["CUSIP"])
    return df["Average Return"].corr(df["Volatility"])